In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os     
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [51]:
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.init
from keras.preprocessing.image import load_img
import gc
gc.collect()
torch.cuda.empty_cache()

In [52]:
root = '/kaggle/input/lottea/shape'

In [53]:
#constrains
WIDTH = 256
HEIGHT = 256
IMAGE_SIZE=(WIDTH,HEIGHT)
CHANNELS = 3


device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [89]:
ft = nn.Sequential(
            nn.Conv2d(3,64,(3,3), padding=1), # (256,256,64)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(128,128,64)
            
            nn.Conv2d(64,128,(3,3), padding=1),#(128,128,128)
            nn.ReLU(True),    
            nn.MaxPool2d(kernel_size=2, stride=2),#(64,64,128)
    
            nn.Conv2d(128,256,(3,3), padding=1),#(64,64,256)
            nn.ReLU(True),
            nn.Conv2d(256,256,(3,3), padding=1),#(64,64,256)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(32,32,256)
    
            nn.Conv2d(256,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.Conv2d(512,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(16,16,512)
    
            nn.Conv2d(512,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.Conv2d(512,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(8,8,512)
        
            
        )

class model(nn.Module):
    def __init__(self, features,num_classes=3):
        super(model, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((8, 8))
        self.features = features
        
        self.classifier = nn.Sequential(
            nn.Linear(512*8*8,4096),
            nn.ReLU(True),
            nn.Linear(4096,4096),
            nn.ReLU(True),
            nn.Linear(4096,3),
        )#FC layer
        
        

    def forward(self, x):
        #print(x.shape)
        x = self.features(x) #Convolution 
        x = self.avgpool(x) # avgpool
        #print(x.shape)
        x = x.view(x.size(0), -1) #
        #print(x.shape)
        x = self.classifier(x) #FC layer
        
       
        return x

m = model(ft).to(device)
m

model(
  (avgpool): AdaptiveAvgPool2d(output_size=(8, 8))
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2

In [90]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 16

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)

trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [91]:
from torchvision.datasets import ImageFolder 

train_dataset = ImageFolder(root = root,transform = trans)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


In [92]:
## define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)

In [58]:
train_dataset.class_to_idx

{'bag': 0, 'bottle': 1, 'box': 2}

In [93]:
total_batch = len(train_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in (train_loader):
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = m(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.995104194
[Epoch:    2] cost = 0.892545044
[Epoch:    3] cost = 0.887405813
[Epoch:    4] cost = 0.841026664
[Epoch:    5] cost = 0.818116248
[Epoch:    6] cost = 0.80626148
[Epoch:    7] cost = 0.746116042
[Epoch:    8] cost = 0.73065418
[Epoch:    9] cost = 0.675860345
[Epoch:   10] cost = 0.583084881
[Epoch:   11] cost = 0.472981483
[Epoch:   12] cost = 0.37465322
[Epoch:   13] cost = 0.335753977
[Epoch:   14] cost = 0.227663055
[Epoch:   15] cost = 0.188582912
Learning Finished!


In [94]:
torch.save(m,'save8.pt')

In [95]:
m=torch.load('/kaggle/working/save8.pt')

In [96]:
from torch.utils.data import DataLoader
from torchvision.io import read_image

In [97]:
trans2 = transforms.Compose([transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [98]:
import glob

path = "./*"
file_list = glob.glob('/kaggle/input/lottea/test/*')
file_list[:10]


['/kaggle/input/lottea/test/pqfzpfftvh.jpg',
 '/kaggle/input/lottea/test/aduzytppm.jpg',
 '/kaggle/input/lottea/test/llhvthqzl.jpg',
 '/kaggle/input/lottea/test/cetgibur.jpg',
 '/kaggle/input/lottea/test/gksljyjdu.jpg',
 '/kaggle/input/lottea/test/cruqoedwo.jpg',
 '/kaggle/input/lottea/test/frnkicftfp.jpg',
 '/kaggle/input/lottea/test/taivqvca.jpg',
 '/kaggle/input/lottea/test/tfzbhtce.jpg',
 '/kaggle/input/lottea/test/xzcaspjcue.jpg']

In [99]:

class DataSet:
    
    def __init__(self, root):
        self.images = []
        self.label = 0
        file_list = glob.glob('/kaggle/input/lottea/test/*')
        file_list.sort()
        for i in trange(len(file_list)):
            self.images.append(read_image(file_list[i]))
    def __len__(self):
    
        return len(self.images)
    def __getitem__(self, idx):
        """ Here we have to return the item requested by `idx`
            The PyTorch DataLoader class will use this method to make an iterable for
            our training or validation loop.
        """
        img = self.images[idx]
        img = img.unsqueeze(0)
        return img.float().to(device)
    def setlabel(self,l):
        self.label = l
 

In [101]:
from tqdm import trange

In [69]:
ds = DataSet('/kaggle/input/lottea/test/')

100%|██████████| 19728/19728 [01:41<00:00, 195.14it/s]


In [109]:
prediction = m(read_image('/kaggle/input/lottea/shape/bottle/103/36.jpg').unsqueeze(0).float().to(device))
print(prediction)
print(torch.argmax(prediction).item())

tensor([[-383.9514,  204.8416,  -35.1059]], device='cuda:0',
       grad_fn=<AddmmBackward>)
1


In [106]:
result = []

with torch.no_grad():
    for i in trange (ds.__len__()):
    
        prediction = m(ds.__getitem__(i)).squeeze()
        result.append(torch.argmax(prediction).item())

    prediction_np = np.array(result)
    print(result)
    file=pd.read_csv('/kaggle/input/lottea/sample_submission.csv')
    file['predictions'] =result
    file.to_csv('result.csv', index=False)

100%|██████████| 19728/19728 [02:21<00:00, 139.91it/s]


[1, 1, 1, 2, 1, 1, 1, 2, 1, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1, 0, 0, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 2, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 2, 1, 0, 1, 1, 2, 0, 1, 2, 0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 0, 0, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 0, 1, 2, 1, 2, 1, 0, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 0, 1, 0, 1, 1, 0, 

In [ ]:
!pip install tqdm